In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import time
import datetime
from sklearn.metrics import mean_absolute_error, mean_squared_error
# Parameter windowing
input_width = 60*4
label_width = 60
shift = label_width
total_window_size = input_width + shift
OUT_STEPS = label_width
# Definisikan irisan untuk input dan label
input_slice = slice(0, input_width)
label_start = total_window_size - label_width
labels_slice = slice(label_start, None)
train_df = None


In [2]:
def  import_data(file_path='D:\\KULIAH\\teep\AI\\dataset\\52 cells\\CellReports.csv'):
    
    df_begin=pd.read_csv(file_path)
    df_1=df_begin.copy()
    df_1.drop(columns=['RRU.PrbAvailDl', 'RRU.PrbAvailUl','RRU.PrbTotUl','RRU.PrbTotDl','RRC.ConnMax'], inplace=True)
    convert_time=pd.to_datetime(df_1['timestamp'], unit='s',origin='unix')
    df_1.insert(df_1.columns.get_loc('timestamp') + 1, 'datetime_column', convert_time)
    df_1.set_index('datetime_column', inplace=True)
    df_1.drop(columns=['timestamp'], inplace=True)
    
    corr_matrix = df_1.corr(numeric_only=True)[['RRU.PrbUsedDl', 'RRU.PrbUsedUl']]
    filtered_corr = corr_matrix[['RRU.PrbUsedDl', 'RRU.PrbUsedUl']]

    # Find columns where correlation with 'RRU.PrbUsedDl' or 'RRU.PrbUsedUl' is greater than 0.5
    columns_with_high_corr = filtered_corr.index[(filtered_corr['RRU.PrbUsedDl'] > 0.5) & (filtered_corr['RRU.PrbUsedUl'] > 0.5)].tolist()

    seleted_columns = ['Viavi.Cell.Name']+ columns_with_high_corr 
    df_2= df_1[seleted_columns].copy()
    cell_name= df_2['Viavi.Cell.Name'].unique()
    df= df_2.copy()
    
    return df, cell_name

In [3]:
def standardize_data(data, train_df, isoutput=True, column_output=['RRU.PrbUsedDl', 'RRU.PrbUsedUl']):
    if isoutput:
        mean = train_df[column_output].mean().values
        std = train_df[column_output].std().values
    else:
        mean = train_df.mean().values
        std = train_df.std().values

    # Reshape for broadcasting with 2D matrix
    mean = mean.reshape(1, -1)
    std = std.reshape(1, -1)

    return (data - mean) / std

def inverse_standardize_data(data, train_df, isoutput=True, column_output=['RRU.PrbUsedDl', 'RRU.PrbUsedUl']):
    if isoutput:
        mean = train_df[column_output].mean().values
        std = train_df[column_output].std().values
    else:
        mean = train_df.mean().values
        std = train_df.std().values

    # Reshape for broadcasting with 2D matrix
    mean = mean.reshape(1, -1)
    std = std.reshape(1, -1)

    return (data * std) + mean


In [4]:
def make_windows(data_x,data_y, total_window_size, input_slice, labels_slice):
    x = []
    y = []
    for i in range(len(data_x) - total_window_size + 1):
        window_x = data_x[i:i+total_window_size]
        x.append(window_x[input_slice])

    for i in range(len(data_y) - total_window_size + 1):
        window_y= data_y[i:i+total_window_size]
        y.append(window_y[labels_slice])

    return np.array(x), np.array(y)


In [5]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import time

def tensorflow_cnn(X_train_scaled, Y_train_scaled, X_validation_scaled, Y_validation_scaled, 
                    learning_rate, target_error, max_epochs, max_sampel_batch,
                    patience, save_best_model_path, validation_data=False, load_model=None, out_steps=OUT_STEPS):
    global model

    class MAEStopCallback(tf.keras.callbacks.Callback):
        def __init__(self, threshold):
            super(MAEStopCallback, self).__init__()
            self.threshold = threshold

        def on_epoch_end(self, epoch, logs=None):
            if logs['mae'] < self.threshold:
                print(f"\nMAE reached below {self.threshold}. Stopping training.")
                self.model.stop_training = True

                
    input_width = X_train_scaled.shape[1]
    CONV_WIDTH = input_width  # Define the width of the convolutional window
    num_features = X_train_scaled.shape[2]
    num_output = Y_train_scaled.shape[2]
    out_steps = out_steps

    model = tf.keras.models.Sequential()
    if load_model is None:
        model.add(tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :], input_shape=(input_width, num_features)))
        model.add(tf.keras.layers.Conv1D(120, activation='relu', kernel_size=CONV_WIDTH, padding='same'))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Conv1D(120, activation='relu', kernel_size=int(CONV_WIDTH/2),padding='same') )
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Conv1D(120, activation='relu', kernel_size=int(CONV_WIDTH),padding='valid') )
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dense(out_steps * num_output, kernel_initializer=tf.initializers.zeros()))
        model.add(tf.keras.layers.Reshape([out_steps, num_output]))
    else:
        model = tf.keras.models.load_model(load_model)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse', 'mae', 'mape', tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    mae_stop_callback = MAEStopCallback(threshold=target_error)

    checkpoint_callback = ModelCheckpoint(
        save_best_model_path,
        monitor='val_loss',     
        mode='min',         
        save_best_only=True, 
        verbose=1            
    )

    early_stopping_callback = EarlyStopping(
        monitor='val_loss',     
        mode='min',         
        patience=patience,    
        restore_best_weights=True,
        verbose=1            
    )

    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=int(2), min_lr=0.00001, verbose=1)

    time_start = time.time()
    if validation_data:
        model.fit(X_train_scaled, Y_train_scaled, epochs=max_epochs, batch_size=max_sampel_batch,  
                  callbacks=[mae_stop_callback, checkpoint_callback, early_stopping_callback, reduce_lr], 
                  validation_data=(X_validation_scaled, Y_validation_scaled), validation_batch_size=max_sampel_batch)
    else:
        model.fit(X_train_scaled, Y_train_scaled, epochs=max_epochs, batch_size=max_sampel_batch, 
                  callbacks=[mae_stop_callback, checkpoint_callback, early_stopping_callback, reduce_lr])
    
    print("time computation seconds: ", time.time() - time_start)
    
    loss, MSE, MAE, RMSE, MAPE = model.evaluate(X_train_scaled, Y_train_scaled)
    print("loss: ", loss, "MSE: ", MSE, "MAE: ", MAE, "RMSE: ", RMSE, "MAPE: ", MAPE)
    
    return model, loss, MSE, MAE, RMSE, MAPE


In [6]:
def compute_metrics(actual_df, predictions_df):
    mae = mean_absolute_error(actual_df, predictions_df)
    mse = mean_squared_error(actual_df, predictions_df)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((actual_df - predictions_df) / actual_df)) * 100
    return mae, mape, mse, rmse

def process_predictions(model, x_scaled, actual_df, input_width, label_width, columns):
    predictions = model.predict(x_scaled[slice(None, None, label_width), :, :])
    predictions_reshaped = predictions.reshape(-1, len(columns))
    predictions_unscaled = inverse_standardize_data(predictions_reshaped, isoutput=True, train_df=train_df)
    predictions_df = pd.DataFrame(predictions_unscaled, columns=columns, index=actual_df[input_width:].index)
    return predictions_df
def compute_error( x_scaled, actual_df, columns, model):
    
    # Process predictions for training and validation sets
    predictions_df = process_predictions(model, x_scaled, actual_df, input_width, label_width, columns)

    display(pd.concat((predictions_df.rename(columns={columns[0]: '%s_predict'%columns[0], columns[1]: '%s_predict'%columns[1]}), actual_df[input_width:]), axis=1))
    mae, mape, mse, rmse = compute_metrics(actual_df[input_width:], predictions_df)
    
    print(f"MAE: {mae}, MAPE %: {mape}, MSE: {mse}, RMSE: {rmse}")

In [7]:

def plot_predictions(val_df, x_val_scaled, model, val_scaled, input_width, label_width, num_features):
    num_output = 2


    # Predictions for validation data
    val_predictions = model.predict(x_val_scaled[slice(None,None,label_width), :, :])
    predictions_reshaped = val_predictions.reshape(-1, num_output)
    predictions_unscaled = inverse_standardize_data(predictions_reshaped, isoutput=True, train_df=train_df)
    val_predictions_df = pd.DataFrame(predictions_unscaled, columns=['RRU.PrbUsedDl', 'RRU.PrbUsedUl'], index=val_df.index)

    # Generate new predictions
    val_new = val_scaled.tail(input_width).to_numpy()
    val_new = val_new.reshape(1, -1, num_features)
    generate_predictions = model.predict(val_new)
    generate_predictions = inverse_standardize_data(generate_predictions.reshape(-1, num_output), isoutput=True, train_df=train_df)
    generate_predictions_df = pd.DataFrame(generate_predictions, columns=['RRU.PrbUsedDl', 'RRU.PrbUsedUl'], index=val_df.tail(label_width).index + pd.DateOffset(hours=1))

    # Combine predictions
    all_predictions = pd.concat([val_predictions_df, generate_predictions_df], axis=0)
    inputs_graph = val_df[['RRU.PrbUsedDl', 'RRU.PrbUsedUl']].iloc[-label_width * 3:-label_width]
    labels_graph = val_df[['RRU.PrbUsedDl', 'RRU.PrbUsedUl']].iloc[-label_width:]
    predictions_graph = all_predictions.loc[labels_graph.index]
    new_predictions_graph = all_predictions.loc[generate_predictions_df.index]

    # Plotting
    plt.figure(figsize=(16, 8))
    for n, feature in enumerate(all_predictions.columns):
        plt.subplot(len(all_predictions.columns), 1, n + 1)
        plt.plot(inputs_graph.index, inputs_graph[feature], label='Inputs for orange', marker='.', zorder=-100, markersize=5)
        plt.plot(labels_graph.index, labels_graph[feature], label='actual output for orange and inputs for red', marker='.', zorder=-100, c='#2ca02c')
        plt.scatter(predictions_graph.index, predictions_graph[feature], marker='X', edgecolors='k', label='Prediction', c='#ff7f0e', s=int(32 * 1.5))
        plt.scatter(new_predictions_graph.index, new_predictions_graph[feature], marker='+', label='New prediction', c='#FF012D', s=int(32 * 1.5))
        plt.ylabel(feature)
        plt.legend()

    plt.subplots_adjust(hspace=0.2, top=1)
    plt.show()


In [8]:
def running_program(my_df,cell_name, index_cell, name_file, name_file_before):
    global train_df
    df_1= my_df[my_df['Viavi.Cell.Name']==cell_name[index_cell]].drop(columns=['Viavi.Cell.Name']).copy()
    df=df_1.astype(float).copy()
    n = len(df)
    train_df =df.loc[:'2023-01-07 00:00:00'].iloc[:-1]
    val_df = df['2023-01-07 00:00:00':]
    num_features = df.shape[1]
    train_scaled = standardize_data(train_df, isoutput=False, train_df=train_df)
    val_scaled= standardize_data(val_df, isoutput=False, train_df=train_df)
    # Membuat windowed dataset untuk set pelatihan, validasi, dan pengujian

    x_train_scaled, y_train_scaled = make_windows(train_scaled.to_numpy(), train_scaled[['RRU.PrbUsedDl', 'RRU.PrbUsedUl']].to_numpy(),total_window_size, input_slice, labels_slice)
    x_val_scaled, y_val_scaled = make_windows(val_scaled.to_numpy(), val_scaled[['RRU.PrbUsedDl', 'RRU.PrbUsedUl']].to_numpy(),total_window_size, input_slice, labels_slice)

    if index_cell==0:
        model, loss, MSE, MAE, RMSE,  MAPE  = tensorflow_cnn(x_train_scaled, y_train_scaled, x_val_scaled, y_val_scaled,
                                                        learning_rate=0.001, target_error=0.001,  max_epochs=10, max_sampel_batch=60, 
                                                        patience=6,  save_best_model_path = name_file, 
                                                        validation_data=True, load_model=None, out_steps=OUT_STEPS)
        model.summary()
        model.save(name_file)
    else:
        model, loss, MSE, MAE, RMSE,  MAPE  = tensorflow_cnn(x_train_scaled, y_train_scaled, x_val_scaled, y_val_scaled,
                                                        learning_rate=0.001, target_error=0.001,  max_epochs=10, max_sampel_batch=60, 
                                                        patience=6,  save_best_model_path = name_file, 
                                                        validation_data=True, load_model=name_file_before, out_steps=OUT_STEPS)
    
    model = tf.keras.models.load_model('%s'%name_file)
    columns = ['RRU.PrbUsedDl', 'RRU.PrbUsedUl']
    output_actual_train = train_df[columns]
    output_actual_val = val_df[columns]
    print("Training Metrics:")
    compute_error(x_train_scaled,  output_actual_train,columns, model)
    print("\nValidation Metrics:")
    compute_error(x_val_scaled,  output_actual_val,columns, model)
    #print("Training Plot:")
    #plot_predictions( output_actual_train[input_width:], x_train_scaled, model, train_scaled, input_width, label_width, num_features)
    #print("\nValidation (test) Plot:")
    #plot_predictions( output_actual_val[input_width:], x_val_scaled, model, val_scaled, input_width, label_width, num_features)
    


In [9]:
df_start, cell_name= import_data(file_path='D:\\KULIAH\\teep\AI\\dataset\\52 cells\\CellReports.csv')
now = datetime.datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M")
skip_indices = [26,29, 31, 37, 40]
for index in range(0,52):
    print(index)
    if index in skip_indices:
        print(index)
        continue
    cells_name= df_start['Viavi.Cell.Name'].unique()
    df_filtered = df_start[df_start['Viavi.Cell.Name'] == cells_name[index]]
    df_no_duplicates = df_filtered.loc[~df_filtered.index.duplicated()]
    name_file='1b_%s_%s.hdf5'%(timestamp, index+1)
    name_file_before='1b_%s_%s.hdf5'%(timestamp, index)
    if index in np.array(skip_indices)+1:
        name_file_before='1b_%s_%s.hdf5'%(timestamp, index-1)
    print("name_file: ", name_file)
    print("name_file_before: ", name_file_before)
    print("Cell Name: ", cell_name[index])
    running_program(my_df=df_no_duplicates, cell_name=cell_name, index_cell=index, name_file=name_file, name_file_before=name_file_before)

0
name_file:  1b_20240724_1930_1.hdf5
name_file_before:  1b_20240724_1930_0.hdf5
Cell Name:  S1/B2/C1
Epoch 1/10
148/148 [==============================] - ETA: 0s - loss: 0.7138 - mse: 0.7138 - mae: 0.5849 - mape: 203.7811 - rmse: 0.8448
Epoch 1: val_loss improved from inf to 0.71508, saving model to 1b_20240724_1930_1.hdf5
148/148 [==============================] - 10s 48ms/step - loss: 0.7138 - mse: 0.7138 - mae: 0.5849 - mape: 203.7811 - rmse: 0.8448 - val_loss: 0.7151 - val_mse: 0.7151 - val_mae: 0.5666 - val_mape: 163.8734 - val_rmse: 0.8456 - lr: 0.0010
Epoch 2/10
146/148 [============================>.] - ETA: 0s - loss: 0.3562 - mse: 0.3562 - mae: 0.4315 - mape: 175.9941 - rmse: 0.5968
Epoch 2: val_loss did not improve from 0.71508
148/148 [==============================] - 9s 58ms/step - loss: 0.3548 - mse: 0.3548 - mae: 0.4306 - mape: 175.8080 - rmse: 0.5956 - val_loss: 1.5089 - val_mse: 1.5089 - val_mae: 1.0264 - val_mape: 457.1169 - val_rmse: 1.2284 - lr: 0.0010
Epoch 3/10

,RRU.PrbUsedDl_predict,RRU.PrbUsedUl_predict,RRU.PrbUsedDl,RRU.PrbUsedUl
datetime_column,,,,
2022-12-31 20:00:00,98.828106,98.828106,98.333333,98.333333
2022-12-31 20:01:00,68.180890,68.180890,69.033333,69.033333
2022-12-31 20:02:00,101.474752,101.474752,100.000000,100.000000
2022-12-31 20:03:00,87.999289,87.999289,89.200000,89.200000
2022-12-31 20:04:00,100.498455,100.498455,100.000000,100.000000
...,...,...,...,...
2023-01-06 23:55:00,98.500056,98.500056,98.066667,98.066667
2023-01-06 23:56:00,97.785416,97.785416,97.233333,97.233333
2023-01-06 23:57:00,93.424818,93.424818,93.816667,93.816667


MAE: 1.1127609594863739, MAPE %: 1.5196172249586, MSE: 2.4034126635420385, RMSE: 1.5502943796395698

Validation Metrics:
1/1 [==============================] - 0s 78ms/step


,RRU.PrbUsedDl_predict,RRU.PrbUsedUl_predict,RRU.PrbUsedDl,RRU.PrbUsedUl
datetime_column,,,,
2023-01-07 04:00:00,97.721701,97.721701,100.000000,100.000000
2023-01-07 04:01:00,87.095948,87.095948,96.200000,96.200000
2023-01-07 04:02:00,87.939284,87.939284,93.333333,93.333333
2023-01-07 04:03:00,97.905171,97.905171,100.000000,100.000000
2023-01-07 04:04:00,85.691031,85.691031,94.633333,94.633333
...,...,...,...,...
2023-01-07 15:55:00,83.808352,83.808352,63.333333,63.333333
2023-01-07 15:56:00,88.427297,88.427297,100.000000,100.000000
2023-01-07 15:57:00,88.817537,88.817537,100.000000,100.000000


MAE: 7.425585437531754, MAPE %: 10.117910925331802, MSE: 112.6643031980694, RMSE: 10.61434421893644
1
name_file:  1b_20240724_1930_2.hdf5
name_file_before:  1b_20240724_1930_1.hdf5
Cell Name:  S7/B2/C1
Epoch 1/10
147/148 [============================>.] - ETA: 0s - loss: 0.2131 - mse: 0.2131 - mae: 0.3328 - mape: 114.2985 - rmse: 0.4616
Epoch 1: val_loss improved from inf to 0.52947, saving model to 1b_20240724_1930_2.hdf5
148/148 [==============================] - 11s 69ms/step - loss: 0.2132 - mse: 0.2132 - mae: 0.3329 - mape: 114.2969 - rmse: 0.4617 - val_loss: 0.5295 - val_mse: 0.5295 - val_mae: 0.6155 - val_mape: 194.8352 - val_rmse: 0.7276 - lr: 0.0010
Epoch 2/10
147/148 [============================>.] - ETA: 0s - loss: 0.0340 - mse: 0.0340 - mae: 0.1470 - mape: 54.5139 - rmse: 0.1843
Epoch 2: val_loss improved from 0.52947 to 0.47949, saving model to 1b_20240724_1930_2.hdf5
148/148 [==============================] - 10s 68ms/step - loss: 0.0341 - mse: 0.0341 - mae: 0.1471 - map

...

MAE: 46.45824051526613, MAPE %: inf, MSE: 3354.1051338056614, RMSE: 57.91463661118544
51
name_file:  1b_20240724_1930_52.hdf5
name_file_before:  1b_20240724_1930_51.hdf5
Cell Name:  S4/N77/C3
Epoch 1/10
147/148 [============================>.] - ETA: 0s - loss: 0.2772 - mse: 0.2772 - mae: 0.3791 - mape: 119.6381 - rmse: 0.5265
Epoch 1: val_loss improved from inf to 0.62079, saving model to 1b_20240724_1930_52.hdf5
148/148 [==============================] - 8s 52ms/step - loss: 0.2772 - mse: 0.2772 - mae: 0.3792 - mape: 119.6359 - rmse: 0.5265 - val_loss: 0.6208 - val_mse: 0.6208 - val_mae: 0.6097 - val_mape: 157.0685 - val_rmse: 0.7879 - lr: 0.0010
Epoch 2/10
148/148 [==============================] - ETA: 0s - loss: 0.0557 - mse: 0.0557 - mae: 0.1774 - mape: 60.8668 - rmse: 0.2359
Epoch 2: val_loss did not improve from 0.62079
148/148 [==============================] - 7s 49ms/step - loss: 0.0557 - mse: 0.0557 - mae: 0.1774 - mape: 60.8668 - rmse: 0.2359 - val_loss: 0.6289 - val_mse: 

,RRU.PrbUsedDl_predict,RRU.PrbUsedUl_predict,RRU.PrbUsedDl,RRU.PrbUsedUl
datetime_column,,,,
2022-12-31 20:00:00,24.578139,18.409181,4.55,4.55
2022-12-31 20:01:00,49.860380,40.844553,13.65,13.65
2022-12-31 20:02:00,46.328832,35.606452,40.95,40.95
2022-12-31 20:03:00,31.314605,29.126827,50.05,40.95
2022-12-31 20:04:00,63.994492,51.238319,40.95,36.40
...,...,...,...,...
2023-01-06 23:55:00,100.756504,86.740206,45.50,40.95
2023-01-06 23:56:00,136.781117,132.591698,168.35,127.40
2023-01-06 23:57:00,87.076288,86.973173,136.50,50.05


MAE: 11.8077326848112, MAPE %: inf, MSE: 254.7239811939195, RMSE: 15.960074598632662

Validation Metrics:
1/1 [==============================] - 0s 21ms/step


,RRU.PrbUsedDl_predict,RRU.PrbUsedUl_predict,RRU.PrbUsedDl,RRU.PrbUsedUl
datetime_column,,,,
2023-01-07 04:00:00,63.924363,37.501609,36.40,31.85
2023-01-07 04:01:00,81.355042,81.668768,45.50,40.95
2023-01-07 04:02:00,98.519814,86.638874,59.15,50.05
2023-01-07 04:03:00,12.393432,14.171201,0.00,0.00
2023-01-07 04:04:00,10.808509,19.570840,50.05,45.50
...,...,...,...,...
2023-01-07 15:55:00,131.392885,113.512146,163.80,150.15
2023-01-07 15:56:00,60.887239,57.640460,13.65,9.10
2023-01-07 15:57:00,51.667670,45.399985,136.50,136.50


MAE: 36.43296942344696, MAPE %: inf, MSE: 2276.8348843554154, RMSE: 47.71619100845556
